In [1]:
import sys
sys.path.append('..')

In [2]:
import sentence_embedding_evaluation_german as seeg
import torch
from typing import List

/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## (1) Instantiate your Embedding model
First, you should load your pretrained embedding.

Here we will generate a random embedding for demonstration purposes.

In [3]:
# generate a random embedding
emb_dim = 512
vocab_sz = 128
emb = torch.randn((vocab_sz, emb_dim), requires_grad=False)
emb = torch.nn.Embedding.from_pretrained(emb)
# assert emb.weight.requires_grad == False

## (2) Specify the preprocessing
The `preprocessor` function converts a sentences as string into embedding vectors of numbers.

Here we will convert the input strings with a nonsensical approach into IDs for the Embedding layer.

In [4]:
def preprocesser(batch: List[str], params: dict=None) -> List[List[float]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[float]]
        A list of embedding vectors
    """
    features = []
    for sent in batch:
        try:
            ids = torch.tensor([ord(c) % 128 for c in sent])
        except:
            print(sent)
        h = emb(ids)
        features.append(h.mean(axis=0))
    features = torch.stack(features, dim=0)
    return features

## (3) Training settings

In [5]:
params = {
    'datafolder': '../datasets',
    'batch_size': 64, 
    'num_epochs': 20,
    # 'early_stopping': True,
    # 'split_ratio': 0.2,  # if early_stopping=True
    # 'patience': 5,  # if early_stopping=True
}

## (4) Specify downstream tasks

In [6]:
downstream_tasks = [
    'TOXIC', 'ENGAGE', 'FCLAIM', 'VMWE',
    'OL19-A', 'OL19-B', 'OL19-C',
    'OL18-A', 'OL18-B', 
    'ABSD-1', 'ABSD-2', 'ABSD-3',
    'MIO-S', 'MIO-O', 'MIO-I', 'MIO-D', 'MIO-F', 'MIO-P', 'MIO-A'
]

# downstream_tasks = [
#     'TOXIC', 'ENGAGE', 'FCLAIM', 'VMWE',
#     'OL19-C', 'ABSD-2', 'MIO-S']

## (5) Run experiments

In [7]:
%%time
results = seeg.evaluate(downstream_tasks, preprocesser, **params)

CPU times: user 1min 40s, sys: 12.1 s, total: 1min 52s
Wall time: 1min 52s


## (6) Display results

In [8]:
[(res['task'], res['epochs'], res['train']['num'], res['test']['num']) for res in results]

[('TOXIC', 20, 3244, 944),
 ('ENGAGE', 20, 3244, 944),
 ('FCLAIM', 20, 3244, 944),
 ('VMWE', 20, 6651, 1446),
 ('OL19-A', 20, 3979, 3030),
 ('OL19-B', 20, 3979, 3030),
 ('OL19-C', 20, 1920, 929),
 ('OL18-A', 20, 5008, 3397),
 ('OL18-B', 20, 5008, 3397),
 ('ABSD-1', 20, 19431, 2554),
 ('ABSD-2', 20, 19431, 2554),
 ('ABSD-3', 20, 19431, 2554),
 ('MIO-S', 20, 1799, 1800),
 ('MIO-O', 20, 1799, 1800),
 ('MIO-I', 20, 1799, 1800),
 ('MIO-D', 20, 1799, 1800),
 ('MIO-F', 20, 3019, 3019),
 ('MIO-P', 20, 4668, 4668),
 ('MIO-A', 20, 1799, 1800)]

In [9]:
metric = 'f1'
mode = 'train'

[(res['task'], res[mode][metric]) for res in results]

[('TOXIC', 0.6676942046855734),
 ('ENGAGE', 0.7392108508014796),
 ('FCLAIM', 0.6664611590628853),
 ('VMWE', 0.8279957901067508),
 ('OL19-A', 0.6891178688112591),
 ('OL19-B', 0.678059814023624),
 ('OL19-C', 0.8661458333333333),
 ('OL18-A', 0.6755191693290735),
 ('OL18-B', 0.6647364217252396),
 ('ABSD-1', 0.8337193144974525),
 ('ABSD-2', 0.6797385620915033),
 ('ABSD-3', 0.575883896865833),
 ('MIO-S', 0.5708727070594775),
 ('MIO-O', 0.8387993329627571),
 ('MIO-I', 0.9155086158977209),
 ('MIO-D', 0.9216231239577543),
 ('MIO-F', 0.7843656840013249),
 ('MIO-P', 0.8258354755784063),
 ('MIO-A', 0.7159533073929961)]

In [10]:
metric = 'f1'
mode = 'test'

[(res['task'], res[mode][metric]) for res in results]

[('TOXIC', 0.628177966101695),
 ('ENGAGE', 0.7351694915254238),
 ('FCLAIM', 0.6694915254237288),
 ('VMWE', 0.8236514522821576),
 ('OL19-A', 0.6778877887788779),
 ('OL19-B', 0.6801980198019802),
 ('OL19-C', 0.8557588805166847),
 ('OL18-A', 0.6655872828966736),
 ('OL18-B', 0.6638210185457757),
 ('ABSD-1', 0.8198903680501175),
 ('ABSD-2', 0.6534847298355521),
 ('ABSD-3', 0.52858261550509),
 ('MIO-S', 0.5372222222222223),
 ('MIO-O', 0.8383333333333334),
 ('MIO-I', 0.9155555555555556),
 ('MIO-D', 0.9211111111111111),
 ('MIO-F', 0.7843656840013249),
 ('MIO-P', 0.8258354755784063),
 ('MIO-A', 0.7161111111111111)]

In [11]:
metric = 'f1-balanced'
mode = 'test'

[(res['task'], res[mode][metric]) for res in results]

[('TOXIC', 0.39115239381494443),
 ('ENGAGE', 0.43881675940457504),
 ('FCLAIM', 0.41301020408163264),
 ('VMWE', 0.45164960182025027),
 ('OL19-A', 0.40597782420054634),
 ('OL19-B', 0.2024160282852092),
 ('OL19-C', 0.4611368909512761),
 ('OL18-A', 0.4538943821903819),
 ('OL18-B', 0.20669020010097205),
 ('ABSD-1', 0.45051635111876076),
 ('ABSD-2', 0.26347778040887204),
 ('ABSD-3', 0.03639991371872303),
 ('MIO-S', 0.3486718568685781),
 ('MIO-O', 0.4560290117860381),
 ('MIO-I', 0.47795823665893267),
 ('MIO-D', 0.4794679005205321),
 ('MIO-F', 0.4395767588639317),
 ('MIO-P', 0.45230552622316084),
 ('MIO-A', 0.41728714794431854)]

In [13]:
from collections import Counter
dict(Counter(['a', 'b', 'a']))


{'a': 2, 'b': 1}

In [15]:
import torch
num = torch.tensor([1,2,0,0,1,1])
dict(Counter(num.detach().numpy()))

{1: 3, 2: 1, 0: 2}